In [1]:
import os
import torch
import random
import numpy as np
from PIL import Image

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import transforms

# Re-use your MultiComponentDataset
# (copy-paste your class definition here if needed)
# -------------------------------------------------
# from your_code import MultiComponentDataset

#########################
# 1. Custom Transform   #
#########################

def multi_modal_transform(components_tuple):
    """
    components_tuple: A tuple of 5 PIL images (one for each component),
                      each 64x64, binary (0 or 255).
    Returns: (input_tensor, mask_tensor)
      - input_tensor: FloatTensor of shape (5, 64, 64), values in [0,1].
      - mask_tensor:  LongTensor of shape (64, 64), with 0=background or i=component label.
    """
    # Convert each PIL image to a (1, 64, 64) tensor in [0, 1]
    to_tensor = transforms.ToTensor()
    component_tensors = [to_tensor(img) for img in components_tuple]  # list of (1, 64, 64)

    # Stack along channel dimension -> shape: (5, 64, 64)
    stacked = torch.cat(component_tensors, dim=0)

    # Create a multi-class mask:
    # 0 = background, 1..5 = which component is "on" for each pixel
    # We assume no overlap: if pixel in channel i > 0.5 => class = i+1
    mask = torch.zeros((64, 64), dtype=torch.long)
    for i in range(5):
        mask[stacked[i] > 0.5] = i + 1

    return stacked, mask


class MultiComponentSegmentationDataset(MultiComponentDataset):
    """
    This dataset inherits from your MultiComponentDataset but overrides __getitem__
    to produce (input, mask) for segmentation.
    """
    def __getitem__(self, idx):
        # Get the 5 PIL images as a tuple
        components_tuple = super().__getitem__(idx)  # returns tuple of length 5

        # Apply the custom transform to get (input_tensor, mask_tensor)
        input_tensor, mask_tensor = multi_modal_transform(components_tuple)

        return input_tensor, mask_tensor


NameError: name 'MultiComponentDataset' is not defined